#   <span style="color:#5F9EA0">Progetto Python Alessandro Pontini Matricola: 852793</span>

***

In [1]:
# importo librerie
import pandas as pd
import datetime
import numpy as np

In [2]:
# scrivo directory
dire = '/Users/alessandropontini/Desktop/progetto_python/additional-kiva-snapshot/loans_lenders.csv'
dire1 = '/Users/alessandropontini/Desktop/progetto_python/additional-kiva-snapshot/loans.csv'
dire2 = '/Users/alessandropontini/Desktop/progetto_python/additional-kiva-snapshot/lenders.csv'
dire3 = '/Users/alessandropontini/Desktop/progetto_python/additional-kiva-snapshot/country_stats.csv'

In [3]:
# df temporanei

temp_df3 = pd.DataFrame()
temp_df4 = pd.DataFrame()
temp_df_NA_lenders = pd.DataFrame()
total_lenders_amount = pd.DataFrame()


***

##  <span style="color:red">1) Normalize the loan_lenders table. In the normalized table, each row must have one loan_id and one lender.</span>

***

In [4]:
# leggo il file csv
df_lenders_not_normalized = pd.read_csv(dire)

In [5]:
# impongo index=loan_id per poter usare explode che per ogni loan_id mi prende ogni elemento della lista creata con la strip 
# e mi crea un nuove righe in un nuovo data set
df_lenders_normalized = df_lenders_not_normalized.set_index('loan_id')['lenders'].str.split(pat=',').explode().reset_index()

In [6]:
# mostro risultato
df_lenders_normalized.head()

,loan_id,lenders
0,483693,muc888
1,483693,sam4326
2,483693,camaran3922
3,483693,lachheb1865
4,483693,rebecca3499


***

##  <span style="color:red">2) For each loan, add a column duration corresponding to the number of days between the disburse time and the planned expiration time. If any of those two dates is missing, also the duration must be missing.</span>

***

In [7]:
# leggo il file loans, ma per la quantità di dati che devo leggere
# preferisco leggere le date come tali e non come stringhe.
df_loans = pd.read_csv(dire1, parse_dates=['disburse_time', 'planned_expiration_time'])

In [8]:
# ci sono delle date che non hanno disburse_time > planned_expiration_time, non ha senso un prestito che si estingue
# prima della data di sborso, perciò controllo se ce ne sono e in effetti si ci sono.
# creo un df che li contiene df_temp
temp_df = df_loans.loc[df_loans['disburse_time'] > df_loans['planned_expiration_time']]


In [9]:
temp_df[['disburse_time', 'planned_expiration_time']]

,disburse_time,planned_expiration_time
63,2016-11-30 08:00:00+00:00,2016-10-31 05:40:03+00:00
96,2015-12-07 08:00:00+00:00,2015-11-12 14:30:11+00:00
177,2015-12-07 08:00:00+00:00,2015-11-13 05:00:03+00:00
207,2016-11-30 08:00:00+00:00,2016-10-28 14:30:08+00:00
217,2016-11-30 08:00:00+00:00,2016-10-31 01:40:03+00:00
...,...,...
1419303,2016-07-16 07:00:00+00:00,2016-06-12 04:30:03+00:00
1419304,2016-07-15 07:00:00+00:00,2016-06-14 00:50:04+00:00
1419374,2016-07-15 07:00:00+00:00,2016-06-12 04:40:03+00:00
1419443,2016-07-15 07:00:00+00:00,2016-06-15 13:00:03+00:00


In [10]:
for i in temp_df:
    x = temp_df.loc[:, ('planned_expiration_time')]
    temp_df.loc[:, ('disburse_time')] = temp_df.planned_expiration_time
    temp_df.loc[:, ('planned_expiration_time')] = x

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [11]:
# uso funzione isin per vedere i loan_id che coincidano e cambio df_loan con i nuovi di df_temp corretti
df_loans.loc[df_loans.loan_id.isin(temp_df.loan_id), ['disburse_time', 'planned_expiration_time']] = temp_df[['disburse_time', 'planned_expiration_time']]

In [12]:
# creo una colonna per la duration e faccio differenza, dopo di che mostro il risultato
df_loans['duration'] = df_loans['planned_expiration_time'] - df_loans['disburse_time']
df_loans[['loan_id','duration']]

,loan_id,duration
0,657307,53 days 19:30:06
1,657259,96 days 14:25:07
2,658010,37 days 13:10:05
3,659347,34 days 19:10:02
4,656933,57 days 22:10:02
...,...,...
1419602,988180,39 days 17:00:03
1419603,988213,39 days 08:40:07
1419604,989109,51 days 14:20:04
1419605,989143,63 days 00:50:02


***

## <span style="color:red">3) Find the lenders that have funded at least twice.</span>

***

In [13]:
temp_df = df_loans[['loan_id', 'status']]

In [14]:
temp_df = df_lenders_normalized.merge(temp_df, how='left', left_on='loan_id', right_on='loan_id')

In [15]:
# creo lista temporanea in cui salvo tutti i loan_name con il value_counts che mi conta quante volte è presente
# e poi chiedo alla lista solo quelli >1
temp_df = temp_df.loc[temp_df.status == 'funded']
temp_df = temp_df.pivot_table(index=['lenders'], aggfunc='size').sort_values(ascending=False)

In [16]:
temp_df[temp_df>1]

lenders
 gooddogg1           142349
 trolltech4460       139191
 gmct                122156
 nms                  93283
 themissionbeltco     70151
                      ...  
 sebastien5634            2
 chaoyi5871               2
jacqui4717                2
 naromie9799              2
 liu9607                  2
Length: 993782, dtype: int64

***

## <span style="color:red">4) For each country, compute how many loans have involved that country as borrowers.</span>


***

In [17]:
# suppongo che in loans che country_code identifichi le nazionalità delle persone perciò con una groupby posso raggrupparle
# e con una size so il numero presente per ogni nazione
df_loans.groupby('country_code').size()

country_code
AF     2337
AL     3075
AM    13952
AZ    10172
BA      608
      ...  
XK     2178
YE     4206
ZA      633
ZM     1277
ZW     5513
Length: 95, dtype: int64

***

## <span style="color:red"> 5) For each country, compute the overall amount of money borrowed.</span>

***

In [18]:
# raggruppo in base allo stato di appartenenza scegliendo di mostrare la colonna loan_amount e sommo i risultati.
df_loans.groupby('country_code')['loan_amount'].sum()

country_code
AF     1967950.0
AL     4307350.0
AM    22950475.0
AZ    14784625.0
BA      477250.0
         ...    
XK     3083025.0
YE     3444000.0
ZA     1006525.0
ZM     1978975.0
ZW     5851875.0
Name: loan_amount, Length: 95, dtype: float64

***

## <span style="color:red">6) Like the previous point, but expressed as a percentage of the overall amount lent.</span>

***

In [19]:
temp = df_loans.funded_amount.sum()

In [20]:
df_loans.groupby('country_code')['loan_amount'].sum()/temp*100

country_code
AF    0.174126
AL    0.381119
AM    2.030683
AZ    1.308160
BA    0.042228
        ...   
XK    0.272789
YE    0.304729
ZA    0.089058
ZM    0.175102
ZW    0.517780
Name: loan_amount, Length: 95, dtype: float64

***

## <span style="color:red">7) Like the three previous points, but split for each year (with respect to disburse time).</span>

***

In [21]:
temp_df = df_loans.set_index('disburse_time')

In [22]:
# vedo quante persone all'anno
temp_df.groupby(['country_code', pd.Grouper(freq='Y')]).size()

country_code  disburse_time            
AF            2007-12-31 00:00:00+00:00     408
              2008-12-31 00:00:00+00:00     370
              2009-12-31 00:00:00+00:00     678
              2010-12-31 00:00:00+00:00     632
              2011-12-31 00:00:00+00:00     247
                                           ... 
ZW            2013-12-31 00:00:00+00:00     426
              2014-12-31 00:00:00+00:00    2078
              2015-12-31 00:00:00+00:00     600
              2016-12-31 00:00:00+00:00     808
              2017-12-31 00:00:00+00:00    1079
Length: 740, dtype: int64

In [23]:
# faccio le somme
temp_df.groupby(['country_code', pd.Grouper(freq='Y')])['loan_amount'].sum()

country_code  disburse_time            
AF            2007-12-31 00:00:00+00:00     194975.0
              2008-12-31 00:00:00+00:00     365375.0
              2009-12-31 00:00:00+00:00     585125.0
              2010-12-31 00:00:00+00:00     563350.0
              2011-12-31 00:00:00+00:00     245125.0
                                             ...    
ZW            2013-12-31 00:00:00+00:00     678525.0
              2014-12-31 00:00:00+00:00    1311575.0
              2015-12-31 00:00:00+00:00     723625.0
              2016-12-31 00:00:00+00:00     788600.0
              2017-12-31 00:00:00+00:00    1237600.0
Name: loan_amount, Length: 740, dtype: float64

In [24]:
# eseguo operazioni punto 6 
temp_df2 = df_loans['loan_amount'].sum()

In [25]:
temp_df.groupby(['country_code', pd.Grouper(freq='Y')])['loan_amount'].sum()/temp_df2*100

country_code  disburse_time            
AF            2007-12-31 00:00:00+00:00    0.016503
              2008-12-31 00:00:00+00:00    0.030926
              2009-12-31 00:00:00+00:00    0.049527
              2010-12-31 00:00:00+00:00    0.047683
              2011-12-31 00:00:00+00:00    0.020748
                                             ...   
ZW            2013-12-31 00:00:00+00:00    0.057432
              2014-12-31 00:00:00+00:00    0.111015
              2015-12-31 00:00:00+00:00    0.061250
              2016-12-31 00:00:00+00:00    0.066749
              2017-12-31 00:00:00+00:00    0.104754
Name: loan_amount, Length: 740, dtype: float64

***

## <span style="color:red">8) For each lender, compute the overall amount of money lent.</span>

***

In [26]:
# uso un df temporaneo per immagazzinare la join tra df_lenders_normalized e df_loans con solo i campi loan_id
# e loan_amount che usero per i conti successivamente
temp_df = df_lenders_normalized.merge(df_loans[['loan_id', 'loan_amount']], how='left', on='loan_id')

In [27]:
# salvo in un df temp 2 il raggruppamento secondo count, per contare il numero totale di ogni loan id
temp_df2 = temp_df.groupby('loan_id')['lenders'].count()

In [28]:
temp_df2

loan_id
84         3
85         2
86         3
88         3
89         4
          ..
1444051    1
1444053    1
1444058    1
1444063    1
1444065    1
Name: lenders, Length: 1387432, dtype: int64

In [29]:
# faccio ultima merge per avere nel mio data set accanto alla colonna di loan id la colonna dei lenders_y
# che corrisponde al mio count
temp_df = temp_df.merge(temp_df2, how='left', on='loan_id')

In [30]:
temp_df

,loan_id,lenders_x,loan_amount,lenders_y
0,483693,muc888,1225.0,40
1,483693,sam4326,1225.0,40
2,483693,camaran3922,1225.0,40
3,483693,lachheb1865,1225.0,40
4,483693,rebecca3499,1225.0,40
...,...,...,...,...
28293926,1206425,trogdorfamily7622,325.0,8
28293927,1206425,danny6470,325.0,8
28293928,1206425,don6118,325.0,8
28293929,1206486,alan5175,150.0,2


In [31]:
# creo nuova colonna per segnare le singole quote per ogni loan_id
temp_df['single_amount']= temp_df['loan_amount']/temp_df['lenders_y']

In [32]:
# ora raggruppo per ogni lender
temp_df.groupby('lenders_x')['single_amount'].sum()

lenders_x
 000               1672.618411
 00000             1380.693644
 0002              2472.563566
 00mike00            52.631579
 0101craign0101    2623.565117
                      ...     
zzanita              87.500000
zzcyna7269           55.357143
zzinnia              38.000000
zzmcfate           2287.291955
zzrvmf8538          106.417625
Name: single_amount, Length: 1639026, dtype: float64

***

## <span style='color:red'> 9) For each country, compute the difference between the overall amount of money lent and the overall amount of money borrowed. Since the country of the lender is often unknown, you can assume that the true distribution among the countries is the same as the one computed from the rows where the country is known.</span>

***

## Problema di fondo che abbiamo trovato che il data set estremamente scarno con molti NULL, perciò dovevamo distribuire i soldi borrowed amolti valori NULL. Personalmento ho optato per la distribuzione del totale loan_amount dei null secondo percentuali di nazionalità

***

In [33]:
# inizio aggiungendo ai miei dataset anche le informazioni dei lenders_info
df_lenders_info = pd.read_csv(dire2)

In [34]:
# merge in una temporanea di df_lenders con la normalizzata 
# per avere loan_id, in temp3 ho tutta la cartella sia con null che no
temp_df = df_lenders_info[['permanent_name', 'country_code']].merge(df_lenders_normalized, how='left', left_on='permanent_name', right_on='lenders')

In [35]:
# temp_df2 contiene tutte le righe in cui lenders è presente in loan e non ha lenders == null
temp_df2=temp_df.dropna(subset=['lenders'])

In [36]:
temp_df2

,permanent_name,country_code,loan_id,lenders
16,naresh2074,US,1059280.0,naresh2074
17,naresh2074,US,1055468.0,naresh2074
48,jinxian2217,NaN,1292306.0,jinxian2217
49,sreedevi9583,NaN,1052062.0,sreedevi9583
52,nicole98153893,NaN,1137907.0,nicole98153893
...,...,...,...,...
3449014,bonnie9739,US,810311.0,bonnie9739
3449015,bonnie9739,US,846429.0,bonnie9739
3449016,bonnie9739,US,809520.0,bonnie9739
3449018,cody7091,US,441719.0,cody7091


In [37]:
# aggiungiamo anche la colonna loan_amount per avere un totale dei lenders/loan/country ( potrebbero esserci dei NULL)
temp_df2=temp_df2.merge(df_loans[['loan_id', 'loan_amount']],how='left',on='loan_id')

In [38]:
# metto i null a 0
temp_df2['loan_amount'] = temp_df2['loan_amount'].fillna(0)

In [39]:
# inizio controllando quelli con country valore che esiste, perdo quasi 3000000 righe
df_lenders_loans_country = temp_df2.dropna(subset=['country_code'])

In [40]:
temp_df3['total_lent'] = df_lenders_loans_country.groupby('country_code')['loan_amount'].sum()

In [41]:
total = temp_df3['total_lent'].sum()

In [42]:
# cosi facendo ho percentuale del totale per ogni country che è il punto che mi serve
temp_df3['percentage'] = round(temp_df3['total_lent']/total,10)

In [43]:
# ora controllo gli na
temp_df2['boolean'] = temp_df2['country_code'].isnull()

In [44]:
temp_df_NA_lenders = temp_df2.loc[temp_df2['boolean'] == True]

In [45]:
temp_df_NA_lenders

,permanent_name,country_code,loan_id,lenders,loan_amount,boolean
2,jinxian2217,NaN,1292306.0,jinxian2217,1400.0,True
3,sreedevi9583,NaN,1052062.0,sreedevi9583,650.0,True
4,nicole98153893,NaN,1137907.0,nicole98153893,200.0,True
6,haitang3172,NaN,1291625.0,haitang3172,300.0,True
7,jishi5401,NaN,1060071.0,jishi5401,400.0,True
...,...,...,...,...,...,...
1387388,tricia9236,NaN,699175.0,tricia9236,1450.0,True
1387396,marwan8219,NaN,735870.0,marwan8219,475.0,True
1387399,eric9768,NaN,898756.0,eric9768,225.0,True
1387405,nicole1492,NaN,485083.0,nicole1492,500.0,True


In [46]:
# trovo totale loan amount degli NA country
total_NA = temp_df_NA_lenders['loan_amount'].sum()

In [47]:
total_NA

251447675.0

In [48]:
# li moltiplico per la percentuale dei country che conosco lenders e poi li sommo con i precedenti
temp_df3.total_lenders_amount_with_country = temp_df3.total_lent + (temp_df3.percentage * total_NA)

<ipython-input-48-51ca0a0eb8ad>:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  temp_df3.total_lenders_amount_with_country = temp_df3.total_lent + (temp_df3.percentage * total_NA)


In [49]:
temp_df3

,total_lent,percentage
country_code,,
AD,1200.0,0.000001
AE,1760225.0,0.001930
AF,68075.0,0.000075
AL,16650.0,0.000018
AM,23575.0,0.000026
...,...,...
WF,2100.0,0.000002
XK,3750.0,0.000004
YE,15175.0,0.000017


In [50]:
temp_df3['total_borrowed'] = df_loans.groupby('country_code')['loan_amount'].sum()

In [51]:
# tolgo al totale dei loan i lent
temp_df3['total_borrowed'] = temp_df3['total_borrowed'] - temp_df3['total_lent']

In [52]:
temp_df3['total_borrowed'] = temp_df3['total_borrowed'].fillna(0)

In [53]:
temp_df3['lent_borrowed'] = temp_df3.total_lent - temp_df3.total_borrowed

In [54]:
temp_df3

,total_lent,percentage,total_borrowed,lent_borrowed
country_code,,,,
AD,1200.0,0.000001,0.0,1200.0
AE,1760225.0,0.001930,0.0,1760225.0
AF,68075.0,0.000075,1899875.0,-1831800.0
AL,16650.0,0.000018,4290700.0,-4274050.0
AM,23575.0,0.000026,22926900.0,-22903325.0
...,...,...,...,...
WF,2100.0,0.000002,0.0,2100.0
XK,3750.0,0.000004,3079275.0,-3075525.0
YE,15175.0,0.000017,3428825.0,-3413650.0


***

## <span style='color:red'>10) Which country has the highest ratio between the difference computed at the previous point and the population?</span>

***

In [55]:
# importo data set che necessito
df_country_stat = pd.read_csv(dire3)

In [56]:
temp_df3.reset_index()

,country_code,total_lent,percentage,total_borrowed,lent_borrowed
0,AD,1200.0,0.000001,0.0,1200.0
1,AE,1760225.0,0.001930,0.0,1760225.0
2,AF,68075.0,0.000075,1899875.0,-1831800.0
3,AL,16650.0,0.000018,4290700.0,-4274050.0
4,AM,23575.0,0.000026,22926900.0,-22903325.0
...,...,...,...,...,...
190,WF,2100.0,0.000002,0.0,2100.0
191,XK,3750.0,0.000004,3079275.0,-3075525.0
192,YE,15175.0,0.000017,3428825.0,-3413650.0
193,ZA,509075.0,0.000558,497450.0,11625.0


In [57]:
temp_df = df_country_stat.groupby('country_code')['population'].sum()

In [58]:
temp_df3['population'] = temp_df

In [59]:
#calcolo percentuale su i lent-borrowed e la popolazione, successivamente trovo l'idmax per la percentuale che mi dice essere CANADA
temp_df3['percentage'] = round(temp_df3['lent_borrowed']/temp_df3['population']*100, 2)
temp_df3=temp_df3.reset_index()

In [60]:
max_ratio = temp_df3.loc[temp_df3['percentage'].idxmax()]

In [61]:
max_ratio

country_code               CA
total_lent        7.97568e+07
percentage             435.41
total_borrowed   -7.97068e+07
lent_borrowed     1.59464e+08
population        3.66242e+07
Name: 30, dtype: object

***

## <span style='color:red'>11) Which country has the highest ratio between the difference computed at point 9 and the population that is not below the poverty line?</span>

***

In [62]:
temp_df2 = df_country_stat.groupby('country_code')[['country_code','population','population_below_poverty_line']].head()

In [63]:
temp_df2['population_below_poverty_line'] = temp_df2['population_below_poverty_line']/100

In [64]:
temp_df2['population_below_poverty_line'] = temp_df2['population'] - (temp_df2['population_below_poverty_line']*temp_df2['population'])

In [65]:
temp_df2 = temp_df2.merge(temp_df3[['country_code', 'lent_borrowed']], left_on='country_code', right_on='country_code')

In [66]:
temp_df2['percentage_poverty'] = round(temp_df2['lent_borrowed']/temp_df2['population_below_poverty_line']*100, 2)

In [67]:
max_ratio = temp_df2.loc[temp_df2['percentage_poverty'].idxmax()]

In [68]:
max_ratio

country_code                              CA
population                          36624199
population_below_poverty_line    3.31815e+07
lent_borrowed                    1.59464e+08
percentage_poverty                    480.58
Name: 65, dtype: object

***

## <span style='color:red'>12) For each year, compute the total amount of loans. Each loan that has planned expiration time and disburse time in different years must have its amount distributed proportionally to the number of days in each year. For example, a loan with disburse time December 1st, 2016, planned expiration time January 30th 2018, and amount 5000USD has an amount of 5000USD * 31 / (31+365+30) = 363.85 for 2016, 5000USD * 365 / (31+365+30) = 4284.04 for 2017, and 5000USD * 30 / (31+365+30) = 352.11 for 2018.</span>

***

### Decido di eliminare tutte le colonne con NaN, in particolar modo tutte le colonne che non presentano disburse_time oppure planned_expiration_time

***

In [69]:
temp_df = df_loans.dropna()

In [70]:
temp_df.loc[:,('disburse_time')]=temp_df.loc[:,('disburse_time')].dt.tz_localize(None)

In [71]:
temp_df.loc[:,('planned_expiration_time')]=temp_df.loc[:,('planned_expiration_time')].dt.tz_localize(None)

In [72]:
temp_df['disburse_time']

3         2014-01-17 08:00:00
9         2014-01-10 08:00:00
11        2014-01-15 08:00:00
13        2013-12-26 08:00:00
18        2015-09-18 07:00:00
                  ...        
1419599   2015-11-16 08:00:00
1419600   2015-11-20 08:00:00
1419601   2015-11-19 08:00:00
1419603   2015-11-24 08:00:00
1419604   2015-11-13 08:00:00
Name: disburse_time, Length: 478837, dtype: datetime64[ns]

In [73]:
temp_df['differenza'] = temp_df['planned_expiration_time'].dt.year - temp_df['disburse_time'].dt.year

<ipython-input-73-8c6a6d9a8ff2>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['differenza'] = temp_df['planned_expiration_time'].dt.year - temp_df['disburse_time'].dt.year


In [74]:
temp_df = temp_df[['planned_expiration_time', 'disburse_time', 'differenza', 'loan_amount', 'duration']]

In [75]:
temp_df.head()

,planned_expiration_time,disburse_time,differenza,loan_amount,duration
3,2014-02-21 03:10:02,2014-01-17 08:00:00,0,625.0,34 days 19:10:02
9,2014-03-19 13:40:02,2014-01-10 08:00:00,0,1825.0,68 days 05:40:02
11,2014-02-20 18:30:01,2014-01-15 08:00:00,0,950.0,36 days 10:30:01
13,2014-03-26 22:24:34,2013-12-26 08:00:00,1,300.0,90 days 14:24:34
18,2015-11-03 18:40:05,2015-09-18 07:00:00,0,375.0,46 days 11:40:05


In [76]:
# mi accorgo che ho solo 1 o 0 come differenza perciò preparo programma apposta
temp_df.groupby('differenza').size()

differenza
0    415502
1     63335
dtype: int64

In [77]:
# tengo solo i giorni
temp_df.duration = temp_df.duration.dt.days

In [78]:
def func(k):
        x = round(( k.loan_amount * k.duration)/(31+365+30),2)
        y = k['planned_expiration_time'].year
        return [y,x]

def func2(k):
        lim_inf = datetime.datetime(k.planned_expiration_time.year, 1,1)
        z = k.planned_expiration_time - lim_inf
        z = z.days
        x = round(( k.loan_amount * z)/(31+365+30),2) 
        y = k.planned_expiration_time.year
        lim_sup = datetime.datetime(k.disburse_time.year, 12, 31)
        f = lim_sup - k.disburse_time
        f = f.days
        u = round(( k.loan_amount * f)/(31+365+30),2) 
        v = k.disburse_time.year
        return [[y,x],[v,u]]


In [79]:
temp_df1 = temp_df.loc[temp_df.differenza == 0]
listx = []

In [80]:
# applico funzione per gli 0 anni di differenza
listx = temp_df1.apply(lambda x : func(x), axis=1)

In [81]:
d1 = {'tot' : listx}
df2 = pd.DataFrame(d1)
df3 = df2.tot.apply(pd.Series)
df3.columns = ['year', 'total']

In [82]:
# applico funzione per gli 1 anni 
temp_df1 = temp_df.loc[temp_df.differenza == 1]
listy = []

In [83]:
listy = temp_df1.apply(lambda x : func2(x), axis=1)

In [84]:
# ottengo due df di liste con i dati immagazzinati dal limite superio e dal limite inferiore
d1 = {'tot' : listy}
df2 = pd.DataFrame(d1)
df4 = df2.tot.apply(pd.Series)
df4.columns = ['data1', 'data2']
df2 = df4['data1']
df4 = df4['data2']

In [85]:
d1 = {'tot' : df2}
df2 = pd.DataFrame(d1)
df5 = df2.tot.apply(pd.Series)
df5.columns = ['year', 'total']

In [86]:
d1 = {'tot' : df4}
df2 = pd.DataFrame(d1)
df6 = df2.tot.apply(pd.Series)
df6.columns = ['year', 'total']

In [87]:
# appendo entrambi i data set creati al primo creato
df3 = df3.append(df5, ignore_index=True)
df3 = df3.append(df6, ignore_index=True)

In [88]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)
df3.year = df3.year.astype('int')

In [89]:
df3.groupby('year').sum()

,total
year,
2011,29.93000
2012,74010.36000
2013,1443746.34000
2014,8845883.56000
2015,10590086.70000
2016,11577277.04000
2017,13381987.53000
2018,214480.37000


# TEMPO IMPIEGATO 12 PUNTI : 5.50min 
# COMPUTER UTILIZZATO : MACBOOK PRO 19
# PROCESSORE : 1,4 GHz Intel Core i5 quad-core
# MEMORIA : 8 GB 2133 MHz LPDDR3
# SCHEDA GRAFICA : Intel Iris Plus Graphics 645 1536 MB
